# PINN for a damped mass-spring system

This notebook, mainly written by Ben Tapley and inspired by [this repo](https://github.com/benmoseley/harmonic-oscillator-pinn), uses [physics informed neural networks (PINNs)](https://doi.org/10.1016/j.jcp.2018.10.045) to model a damped mass-spring system.

#### Exercises:

* Run the standard neural network model to learn the dynamics. Experiment with the hyperparameters and see if you can get the model to generalise better outside the domain.
* Train a PINN model on the same data and plot the results. Experiment with hyperparameters and data parameters.
* Train a physics informed neural network but learn the parameters $\mu$ and $k$ (i.e., treat them as unknown parameters). 
* Add noise and repeat the experiments.

In [ ]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    import torch
    import torch.nn as nn
    from tqdm import trange
    import seaborn as sns
except ModuleNotFoundError:
    import os
    if not os.path.exists("requirements.txt"):
        print("Downloading requirements.txt from GitHub...")
        import urllib.request
        url = "https://raw.githubusercontent.com/SINTEF-Digital-Analytics-and-AI/NLDL-tutorial/main/requirements.txt"
        urllib.request.urlretrieve(url, "requirements.txt")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    import matplotlib.pyplot as plt
    import numpy as np
    import torch
    import torch.nn as nn
    from tqdm import trange
    import seaborn as sns
if int(np.__version__.split('.')[0]) >= 2:
    import subprocess
    import sys
    print("NumPy version >= 2 detected. Downgrading to a compatible version...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2"])
    print("Please restart the kernel and rerun the script.")

np.random.seed(1)
torch.random.manual_seed(1)

plt.rcParams['font.size'] = 12
plt.rcParams['lines.markersize'] = 10
plt.rcParams['legend.fontsize'] = 10
colors = sns.color_palette([(0.6,0.8,.8), (1,0.7,0.3), (0.2,0.7,0.2), (0.8,0,0.2), (0,0.4,1), (0.6,0.5,.9), (0.5,0.3,.5)])

### Exact solution and data
Here, we will learn the dynamics of a 1D damped mass-spring system with unit mass, spring constant $k$ and damping coefficient $\mu$:
$$
\dfrac{d^2 x}{d t^2} + \mu \dfrac{d x}{d t} + kx = 0~.
$$

For the underdamped case, $\mu^2 < 4 k~,$ the exact solution is given by 
$$
x(t) = e^{-\delta t}(A \cos(\phi + \omega t)),
$$
where $\omega=\frac{1}{2}\sqrt{4k - \mu^2}$ and the constants $A$ and $\phi$ determine the initial conditions. We will set $A=1$ and $\phi=0$.

Let us turn the above exact solution into a function that we can use to generate a training data set $X_{\mathrm{train}}$ consisting of points $(t_i, x_i)$, where $t_i$ are evenly spaced times on some interval and $x_i=x(t_i)$ is given by the above exact solution. We will also create a test data set $X_{\mathrm{test}}$ on a longer time interval. 

In [ ]:
# ODE parameters
MU = 0.4
K = 4

# Data generation parameters
N_TRAIN = 20
TMAX_TRAIN = 2
NOISE_STD = 0.

N_TEST = 50
TMAX_TEST = 10
_
def exact_solution(t, k=K, mu=MU):
    """Get exact solution to the 1D underdamped harmonic oscillator."""
    assert mu**2 < 4 * k, "System must be underdamped."
    w = np.sqrt(4 * k - mu**2) / 2
    x = torch.exp(-mu / 2 * t) * torch.cos(w * t)
    return x


t_train = torch.linspace(0, TMAX_TRAIN, N_TRAIN + 1).unsqueeze(-1)
t_test = torch.linspace(0, TMAX_TEST, N_TEST + 1).unsqueeze(-1)

x_train = exact_solution(t_train) + NOISE_STD * torch.randn_like(t_train)
x_test = exact_solution(t_test)

plt.figure(figsize=(12,3))
plt.plot(t_test, x_test, "k-", label="Test data (exact sol.)")
plt.plot(t_train, x_train, color = colors[0], linestyle="none", marker=".", label=f'Training data')
plt.title("Data")
plt.legend()
plt.show()

### Learn the dynamics with a purely data-driven model
Now we will set up a neural network $x_\theta(t)$ to learn the dynamics $x(t)$ from the training data. We hope that it will generalise well to the test data, which is the exact solution over a longer time span. 

We do this by training the neural network on a loss function that minimises the mean squared error (MSE) between the predicted solution $x_{\theta}(t_i)$ and the observed (exact solution) values $(t_i, x_i)\in X_{\mathrm{train}}$:

$$L_{\mathrm{data}} = \sum_{(t_i, x_i)\in X_{\mathrm{train}}}\|x_{\theta}(t_i) - x_i\|^2$$

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, output_size=1):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        return self.network(x)

def train_nn(model, t_train, x_train, nepochs=10000, learning_rate=5e-3):
    mse = nn.MSELoss()
    torch.manual_seed(123)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses = []
    
    with trange(nepochs, desc="Training the model") as pbar:
        for i in range(nepochs):
            optimizer.zero_grad()

            # Compute the loss
            x_pred = model(t_train)
            loss = mse(x_pred, x_train)

            # Backward pass and update parameters
            loss.backward()
            optimizer.step()

            # Log the loss value
            losses.append(loss.item())
            if i % 100 == 0 or i == nepochs - 1:
                pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    # Plot the loss curve
    plt.figure(figsize=(7, 4))
    plt.plot(losses)
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.show()


model_nn = NeuralNet()
train_nn(model_nn, t_train=t_train, x_train=x_train)

plt.figure(figsize=(12, 3))
plt.plot(t_test, x_test, "k-", label="Exact solution")
plt.plot(t_train, x_train, color = colors[0], linestyle="none", marker=".", label=f'Training data')
plt.plot(t_test, model_nn(t_test).detach().numpy(), "r-", label="Standard NN")
plt.legend()
plt.show()

### Learn the dynamics with a physics informed neural network
Now we will train the physics informed neural network (PINN). This is done in exactly the same way as before, but we augment the loss function with the "physics loss". That is,

$$L_{\mathrm{PINN}}=L_{\mathrm{data}} + \lambda L_{\mathrm{phys.}}$$
where $\lambda$ is a hyperparameter, and
$$L_{\mathrm{phys.}} = \sum_{t_i\in X_{\mathrm{phys.}}}\|\ddot{x}_{\theta}(t_i)+ \mu\,\dot{x}_{\theta}(t_i) + k\,x_{\theta}(t_i)\|^2,$$
where $X_{\mathrm{phys.}}$ are a set of times, chosen by us, that we choose to evaluate the *physics loss* on.

In [ ]:
# choose points to evaluate the physics loss with
TMAX_PHYS = 7
N_PHYS = 200

t_phys = np.random.uniform(0, TMAX_PHYS, N_PHYS)

model_pinn = NeuralNet(1, 32, 1)

def time_derivative(x, t):
    """Returns the time derivative of x at times t using automatic differentiation.
    Example: 
        t = torch.linspace(0, 1, 10)
        x = model(t)
        xdot = time_derivative(x, t)"""
    xdot = torch.autograd.grad(x, t, torch.ones_like(x), create_graph=True)[0]
    return xdot

def train_pinn(model, t_train, x_train, t_phys, nepochs=10000, learning_rate=5e-3):
    lambda_ = 1e-1
    mse = nn.MSELoss()
    t_phys = torch.tensor(t_phys, requires_grad=True, dtype=torch.float32).reshape(-1, 1)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses = []

    with trange(nepochs, desc="Training PINN") as pbar:
        for i in range(nepochs):
            optimizer.zero_grad()

            # Compute data loss
            x_pred = model(t_train)
            loss = mse(x_pred, x_train)

            # Compute physics loss
            x_phys = model(t_phys)
            xdot = time_derivative(x_phys, t_phys)
            xddot = time_derivative(xdot, t_phys)
            ode_residual = xddot + MU * xdot + K * x_phys
            loss += lambda_ * torch.mean(ode_residual**2)

            # Backward pass and parameter update
            loss.backward()
            optimizer.step()

            # Log loss value
            losses.append(loss.item())
            if i % 100 == 0 or i == nepochs - 1:
                pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    # Plot the loss curve
    plt.figure(figsize=(7, 4))
    plt.plot(losses)
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve for PINN')
    plt.show()


# Training the PINN
train_pinn(model_pinn, t_train, x_train, t_phys)

plt.figure(figsize=(12, 3))
plt.plot(t_test, x_test, "k-", label="Exact solution")
plt.plot(t_train, x_train, color = colors[0], linestyle="none", marker=".", label='Training data')
plt.plot(t_phys, 0 * t_phys, color = colors[1], linestyle="none", marker=".", label="Physics loss points")
plt.plot(t_test, model_nn(t_test).detach().numpy(), "r-", label="Standard NN")
plt.plot(t_test, model_pinn(t_test).detach().numpy(), "b-", label="PINN")
plt.xlabel("$t$")
plt.ylabel("$x$")
plt.legend(bbox_to_anchor=(0.5, -0.2), loc='upper center', ncol=5)
plt.show()

### Learning the parameters $\mu$ and $k$

What we have just done was solve the forward problem. Traditional numerics usually beats PINNs in terms of cost and accuracy. However, the scenario where PINNs outperforms traditional numerics is when we have *partial* knowledge of the governing equations. 

Say that the parameters $\mu$ and $k$ are unknown. Using the data (which encodes the values of $\mu$ and $k$) together with knowledge that the governing ODE is partially known, we can treat $\mu$ and $k$ as learnable parameters and adopt the same approach as before. 

In [ ]:
class PINN(nn.Module):

    def __init__(self, input_size=1, hidden_size=32, output_size=1):
        super().__init__()

        # Tell the pytorch module to learn the parameters:
        self.mu = nn.Parameter(torch.tensor(0.0, requires_grad=True))
        self.k = nn.Parameter(torch.tensor(0.0, requires_grad=True))

        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        return self.network(x)


from tqdm import trange
import matplotlib.pyplot as plt

def train_pinn(model, t_train, x_train, t_phys, nepochs=10000, learning_rate=5e-3):
    lambda_ = 1e-1
    mse = nn.MSELoss()
    t_phys = torch.tensor(t_phys, requires_grad=True, dtype=torch.float32).reshape(-1, 1)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses = []

    with trange(nepochs, desc="Training PINN with parameters") as pbar:
        for i in range(nepochs):
            optimizer.zero_grad()

            # Compute data loss
            x_pred = model(t_train)
            loss = mse(x_pred, x_train)

            # Compute physics loss (parameters mu and k in the ODE)
            mu = model.mu
            k = model.k
            x_phys = model(t_phys)
            xdot = time_derivative(x_phys, t_phys)
            xddot = time_derivative(xdot, t_phys)
            ode_residual = xddot + mu * xdot + k * x_phys
            loss += lambda_ * torch.mean(ode_residual**2)

            # Backward pass and parameter update
            loss.backward()
            optimizer.step()

            # Log loss value
            losses.append(loss.item())
            if i % 100 == 0 or i == nepochs - 1:
                pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    # Plot the loss curve
    plt.figure(figsize=(7, 4))
    plt.plot(losses)
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve for PINN with Parameters')
    plt.show()


# Define the model and train
model_pinn2 = PINN(1, 32, 1)
train_pinn(model_pinn2, t_train, x_train, t_phys)

print(
    f"learned μ = {model_pinn2.mu.item():.3f}, exact μ = {MU}\nlearned k = {model_pinn2.k.item():.3f}, exact k = {K}"
)

plt.figure(figsize=(10, 3))
plt.plot(t_test, x_test, "k-", label="Exact solution")
plt.plot(t_train, x_train, color = colors[0], linestyle="none", marker=".", label='Training data')
plt.plot(t_phys, 0 * t_phys, color = colors[1], linestyle="none", marker=".", label="Physics loss points")
plt.plot(t_test, model_nn(t_test).detach().numpy(), "r-", label="Standard NN")
plt.plot(t_test, model_pinn2(t_test).detach().numpy(), "b-", label="PINN")
plt.xlabel("$t$")
plt.ylabel("$x$")
plt.legend(bbox_to_anchor=(0.5, -0.2), loc='upper center', ncol=5)
plt.show()